Generator generate the fake image from noise and descriminator used to check eihet the image is real or fake.
Generator we perform upsample ane in discriminator we perform normal classiifcation(downsample).
CONV2DTranspose is as upsampling image.
we need two loss functions:
1. for generator ( ccross entropy-
2.for descriminator(real output and fake output)

we need two optimizers
one Adam Optimizer to update the weights for generator and another one for descriminator.
key items:
Noise dimension
to generate images
we pass noise to generator then we pass both fakeotput and real output to discriminator .


In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import cv2

# Load the dataset of face images
faces_dataset = keras.preprocessing.image_dataset_from_directory(
"C:/Users/annem/OneDrive/Desktop/cnn/train/",
validation_split=0.2,
subset="training",
seed=42,
image_size=(64, 64),
batch_size=32,
)

Found 262 files belonging to 17 classes.
Using 210 files for training.


In [2]:


# Define the generator model
def build_generator(latent_dim):
    model = keras.Sequential()
    model.add(layers.Dense(8 * 8 * 256, input_dim=latent_dim))
    model.add(layers.Reshape((8, 8, 256)))
    model.add(layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding="same"))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding="same"))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding="same"))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Conv2D(3, (3, 3), activation="tanh", padding="same"))
    return model

# Define the discriminator model
def build_discriminator(img_shape):
    model = keras.Sequential()
    model.add(layers.Conv2D(64, (3, 3), strides=(2, 2), padding="same", input_shape=img_shape))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dropout(0.4))
    model.add(layers.Conv2D(64, (3, 3), strides=(2, 2), padding="same"))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dropout(0.4))
    model.add(layers.Conv2D(64, (3, 3), strides=(2, 2), padding="same"))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dropout(0.4))
    model.add(layers.Flatten())
    model.add(layers.Dense(1, activation="sigmoid"))
    return model

# Define the GAN model
def build_gan(generator, discriminator):
    discriminator.trainable = False
    model = keras.Sequential()
    model.add(generator)
    model.add(discriminator)
    return model

# Define the loss functions
cross_entropy = keras.losses.BinaryCrossentropy(from_logits=True)

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

# Define the optimizer
generator_optimizer = keras.optimizers.Adam(1e-4)
discriminator_optimizer = keras.optimizers.Adam(1e-4)

# Define the training loop
latent_dim = 100
epochs = 50

generator = build_generator(latent_dim)
discriminator = build_discriminator((64, 64, 3))
gan = build_gan(generator, discriminator)


In [4]:
import matplotlib.pyplot as plt
image_size = 64
#print(f"Epoch {epoch+1}/{epochs}")
for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}")
    for i, batch in enumerate(faces_dataset):
       # print(i)
        #print(len(batch))
        noise = tf.random.normal([batch[0].shape[0], latent_dim])
        # Generate fake images
        with tf.GradientTape() as tape:
            fake_images = generator(noise, training=True)
            fake_output = discriminator(fake_images, training=True)
            gen_loss = generator_loss(fake_output)
        gradients_of_generator = tape.gradient(gen_loss, generator.trainable_variables)
        generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))

        # Train the discriminator
        with tf.GradientTape() as tape:
            real_output = discriminator(batch[0], training=True)
            fake_output = discriminator(fake_images, training=True)
            disc_loss = discriminator_loss(real_output, fake_output)
        gradients_of_discriminator = tape.gradient(disc_loss, discriminator.trainable_variables)
        discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

        # Print loss values every 100 batches
        if i % 100 == 0:
            print(f"Batch {i}/{len(faces_dataset)} - Gen loss: {gen_loss} - Disc loss: {disc_loss}")

    # Generate a sample of fake images at different ages
    for age in range(20, 80, 10):
        # Generate random noise
        noise = tf.random.normal([1, latent_dim])

        # Generate a fake image at the specified age
        fake_image = generator(noise, training=False)
        fake_image = np.squeeze(fake_image.numpy())
        fake_image = (fake_image + 1) / 2.0 
        file_name=f"C:/Users/annem/OneDrive/Desktop/{age}.png"
        plt.imsave(file_name, fake_image)
#         fig, axs=plt.subplots(1, fake_image, figsize=(15,15))
#         for i in range(fake_image):
#             axs[i].imshow(fake_image[i])
# #             axs[i].axis('off')
# #         plt.show()
        
#         # Rescale pixel values to [0, 1]
#         fake_image = cv2.cvtColor(fake_image, cv2.COLOR_RGB2BGR)  # Convert from RGB to BGR
#         fake_image = (fake_image * 255).astype(np.uint8)  # Rescale pixel values to [0, 255]

#         # Save the fake image
#         cv2.imshow(f"fake_image_{age}.jpg", fake_image)
#         cv2.waitKey(0)
#         cv2.destroyAllWindows()

Epoch 1/50
Batch 0/7 - Gen loss: 0.5297284126281738 - Disc loss: 6.810472011566162
Epoch 2/50
Batch 0/7 - Gen loss: 0.4663059711456299 - Disc loss: 9.340594291687012
Epoch 3/50
Batch 0/7 - Gen loss: 0.4445614516735077 - Disc loss: 6.74867057800293
Epoch 4/50
Batch 0/7 - Gen loss: 0.3661104738712311 - Disc loss: 11.252571105957031
Epoch 5/50
Batch 0/7 - Gen loss: 0.3609847128391266 - Disc loss: 7.508737564086914
Epoch 6/50
Batch 0/7 - Gen loss: 0.36988407373428345 - Disc loss: 4.344683647155762
Epoch 7/50
Batch 0/7 - Gen loss: 0.34205976128578186 - Disc loss: 8.166535377502441
Epoch 8/50
Batch 0/7 - Gen loss: 0.34578636288642883 - Disc loss: 7.831428050994873
Epoch 9/50
Batch 0/7 - Gen loss: 0.31943339109420776 - Disc loss: 4.287395477294922
Epoch 10/50
Batch 0/7 - Gen loss: 0.3161329925060272 - Disc loss: 9.522903442382812
Epoch 11/50
Batch 0/7 - Gen loss: 0.311786949634552 - Disc loss: 9.112005233764648
Epoch 12/50
Batch 0/7 - Gen loss: 0.28062257170677185 - Disc loss: 8.3042964935302